In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ECE1786_Project/

In [2]:
!pip install wandb

import wandb
wandb.login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dhairya18498. Use `wandb login --relogin` to force relogin


True

In [ ]:
wandb.init(project="codet5_base_finetuning", entity="dhairya18498")

In [4]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd
dhairya_df = pd.read_csv('Dataset_Dhairya.csv')
yuchen_df = pd.read_csv('Dataset_Yuchen.csv')

In [7]:
dhairya_df.head()

,text,code
0,Write a python program for time Complexity pl...,import time\nfrom numpy.random import seed\nfr...
1,Write a python function for Stooge Sort\n,"def stoogesort(arr, l, h):\n if l >= h:\n ret..."
2,Write a python function for Recursive Inserti...,"def insertionSortRecursive(arr, n):\n if n ..."
3,Write a python function for Cycle Sort\n,def cycleSort(array):\n writes = 0\n \n fo...
4,Write a python program for BogoSort\n,import random\n\ndef is_sorted(a):\n n = le...


In [8]:
yuchen_df.head()

,text,code
0,write a python program to sort an array in asc...,def bubbleSort(array):\n swapped = False\n ...
1,write a python program to sort an array in des...,def bubbleSort(array):\n swapped = False\n ...
2,write a python program to find the element tha...,def bubbleSort(array):\n swapped = False\n ...
3,write a python program to check if any value a...,def bubbleSort(array):\n swapped = False\n ...
4,write a python program to find the third disti...,def bubbleSort(array):\n swapped = False\n ...


In [9]:
finetuning_dataset = pd.concat([dhairya_df, yuchen_df], ignore_index = True)
finetuning_dataset.head()

,text,code
0,Write a python program for time Complexity pl...,import time\nfrom numpy.random import seed\nfr...
1,Write a python function for Stooge Sort\n,"def stoogesort(arr, l, h):\n if l >= h:\n ret..."
2,Write a python function for Recursive Inserti...,"def insertionSortRecursive(arr, n):\n if n ..."
3,Write a python function for Cycle Sort\n,def cycleSort(array):\n writes = 0\n \n fo...
4,Write a python program for BogoSort\n,import random\n\ndef is_sorted(a):\n n = le...


In [10]:
from sklearn.model_selection import train_test_split
finetuning_train_val,  finetuning_test = train_test_split(finetuning_dataset, test_size = 0.10, random_state = 42)
finetuning_train,  finetuning_val = train_test_split(finetuning_train_val, test_size = 0.10, random_state = 42)

In [11]:
finetuning_val.head()

,text,code
19,Write a python program for Radix Sort\n,"def countingSort(arr, exp1):\n \n n = len..."
22,Write a python function for Insertion Sort\n,def insertionSort(arr):\n \n for i in range...
40,write a python program to sort an array in asc...,def bubbleSort(array):\n swapped = False\n ...
73,write a python program to check if any value a...,def quickSort(array):\n if len(array)> 1:\n...
9,Write a python function for Cocktail Sort\n,def cocktailSort(a):\n n = len(a)\n swap...


In [12]:
from datasets import Dataset
training_set = Dataset.from_pandas(finetuning_train)
validation_set = Dataset.from_pandas(finetuning_val)
test_set = Dataset.from_pandas(finetuning_test)

In [13]:
training_set = training_set.remove_columns([list(training_set.features.keys())[-1]])
training_set

Dataset({
    features: ['text', 'code'],
    num_rows: 72
})

In [14]:
validation_set = validation_set.remove_columns([list(validation_set.features.keys())[-1]])
validation_set

Dataset({
    features: ['text', 'code'],
    num_rows: 8
})

In [15]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base")

model = AutoModelWithLMHead.from_pretrained("Salesforce/codet5-base")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1136: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [16]:
def prepare_data(examples):
    text = examples['text']
    code = examples['code']
    
    model_inputs = tokenizer(text, padding="longest", truncation = True)
    labels = tokenizer(code, padding="longest", truncation = True).input_ids

    labels_with_ignore_index = []
    for labels_example in labels:
      labels_example = [label if label != 0 else -100 for label in labels_example]
      labels_with_ignore_index.append(labels_example)
    
    model_inputs["labels"] = labels_with_ignore_index

    return model_inputs

In [17]:
tokenized_training_set = training_set.map(prepare_data, batched=True, remove_columns=training_set.column_names)
tokenized_validation_set = validation_set.map(prepare_data, batched=True, remove_columns=validation_set.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
labels = tokenized_training_set["labels"][0]
print(tokenizer.decode([label for label in labels if label != -100]))

<s>import itertools
def radixSort(array):
    min_num = min(array)
    if min_num < 0:
        lb = abs(min_num)
        array = [item + lb for item in array]
    n_digits = len(str(max(array)))
    for dgt in range(n_digits):
        buckets = [[] for i in range(10)]
        for num in array:
            idx = (num // (10**dgt)) % 10
            buckets[idx].append(num)
        array = list(itertools.chain(*buckets))
    if min_num < 0:
        lb = abs(min_num)
        array = [item - lb for item in array]
    return array
arr = [9,4,3,5,6,1,2,7]
print(radixSort(arr))




</s>


In [19]:
from transformers import TrainingArguments

args = TrainingArguments(
    "trainer",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=20,
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="wandb"
)

In [20]:
from transformers import default_data_collator

data_collator = default_data_collator

In [21]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

146

In [22]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_training_set,
    eval_dataset=tokenized_validation_set,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [23]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

22

In [24]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 72
  Num Epochs = 20
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 720
  Number of trainable parameters = 222882048
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,No log,2.775501
2,No log,2.012217
3,No log,1.674972
4,No log,1.528241
5,No log,1.413057
6,No log,1.352535
7,No log,1.324801
8,No log,1.281714
9,No log,1.256524
10,No log,1.230487


***** Running Evaluation *****
  Num examples = 8
  Batch size = 2
***** Running Evaluation *****
  Num examples = 8
  Batch size = 2
***** Running Evaluation *****
  Num examples = 8
  Batch size = 2
***** Running Evaluation *****
  Num examples = 8
  Batch size = 2
***** Running Evaluation *****
  Num examples = 8
  Batch size = 2
***** Running Evaluation *****
  Num examples = 8
  Batch size = 2
***** Running Evaluation *****
  Num examples = 8
  Batch size = 2
***** Running Evaluation *****
  Num examples = 8
  Batch size = 2
***** Running Evaluation *****
  Num examples = 8
  Batch size = 2
***** Running Evaluation *****
  Num examples = 8
  Batch size = 2
***** Running Evaluation *****
  Num examples = 8
  Batch size = 2
***** Running Evaluation *****
  Num examples = 8
  Batch size = 2
***** Running Evaluation *****
  Num examples = 8
  Batch size = 2
Saving model checkpoint to trainer/checkpoint-500
Configuration saved in trainer/checkpoint-500/config.json
Model weights saved i

TrainOutput(global_step=720, training_loss=1.1718093660142688, metrics={'train_runtime': 259.5727, 'train_samples_per_second': 5.548, 'train_steps_per_second': 2.774, 'total_flos': 71933150822400.0, 'train_loss': 1.1718093660142688, 'epoch': 20.0})

In [25]:
trainer.save_model("codet5_base_finetuned_on_sorting_algorithms")

Saving model checkpoint to codet5_base_finetuned_on_sorting_algorithms
Configuration saved in codet5_base_finetuned_on_sorting_algorithms/config.json
Model weights saved in codet5_base_finetuned_on_sorting_algorithms/pytorch_model.bin
tokenizer config file saved in codet5_base_finetuned_on_sorting_algorithms/tokenizer_config.json
Special tokens file saved in codet5_base_finetuned_on_sorting_algorithms/special_tokens_map.json


In [26]:
model = AutoModelWithLMHead.from_pretrained("codet5_base_finetuned_on_sorting_algorithms")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1136: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
loading configuration file codet5_base_finetuned_on_sorting_algorithms/config.json
Model config T5Config {
  "_name_or_path": "codet5_base_finetuned_on_sorting_algorithms",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 1,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "label2id": {
    "LABEL_0": 0
 

In [27]:
sample_input = finetuning_test["text"][10]
print('sample_input:', sample_input)
input_ids = tokenizer(sample_input, return_tensors='pt').input_ids
outputs = model.generate(input_ids, max_new_tokens=300)
print('output:\n', tokenizer.decode(outputs[0], skip_special_tokens=True))

sample_input:  Write a python program for Bitonic Sort

output:
 def bitonicSort(arr):
    n = len(arr)
    i = 0
    while i < n and arr[i] > arr[i+1]:
        i += 1
        arr[i], arr[i+1] = arr[i+1], arr[i]
            i += 1
        arr[i], arr[i+1] = arr[i+1], arr[i]
    return arr





In [28]:
sample_input = test_set["text"][5]
print('sample_input:', sample_input)
input_ids = tokenizer(sample_input, return_tensors='pt').input_ids
outputs = model.generate(input_ids, max_new_tokens=300)
print('output:\n', tokenizer.decode(outputs[0], skip_special_tokens=True))

sample_input:  Write a python program for time Complexity plot of Heap Sort

output:
 def heapify(arr, n):
    i_lower_bound, upper_bound = min(arr), max(arr)
    lower_bound = i_lower_bound
    if i_lower_bound < 0:
        lb = abs(i_lower_bound)
        lower_bound = abs(i_lower_bound)
        heapify(arr, n_lower_bound, upper_bound)
    if i_lower_bound < 0:
        return (arr + (upper_bound - lower_bound)
def heapify(arr, n):
    heapify(arr, n, i_lower_bound, upper_bound)
def heapify(arr, i_lower_bound, left):
        heapify(arr, left, right)
    return heapify(arr, left, right)
def heapify(arr, n_lower_bound, upper_bound):
    heapify(arr, left, right):
        heapify(arr, left, right):
    heapify(arr, left, right):
    heapify(arr, left, right, right):
def heapify(arr, i_lower_bound, left):
    heapify(arr, left, right):
    heapify(arr, left, right):
    heapify(arr, left, right):
    heap
